# build_data

we build our data set here.

In [ ]:
#FIXME: in original data, we have pid starts from 0, but here we need to set pid starts from 1

In [17]:
#1. vocabulary construction.
input_path='../../PerGen/code/OpenNMT-py/src_data_full_feat_tf_resplited_review'

# 1.1 read src data.
def read_src_data(file,vocab_dict):
    src=[]
    with open(file,'r') as fin:
        lines = fin.readlines()
        for line in lines:
            line=line.strip().split(' ')
            src.append(line)
            for word in line:
                if word not in vocab_dict:
                    vocab_dict[word]=0
                vocab_dict[word]+=1
    return src,vocab_dict

# 1.2 read tgt data.
def read_tgt_data(file,vocab_dict):
    tgt=[]
    with open(file,'r') as fin:
        lines = fin.readlines()
        for i,line in enumerate(lines):
            line=line.strip().split(' ')
            pid=None
            tgt_line=[]
            for word_pid in line:
#                 print(word_pid)
                word,pid=word_pid.split('￨')

                tgt_line.append(word)
                if word not in vocab_dict:
                    vocab_dict[word]=0
                vocab_dict[word]+=1
            tgt.append({'line':tgt_line,'pid':int(pid)+1})
            
            #FIXME: in original data, we have pid starts from 0, but here we need to set pid starts from 1
            
    return tgt,vocab_dict
def build_vocab(vocab_dict,min_frequency):
    ## we need to filter out those unfrequent words out.
    vocab_index_dict={}
    index_vocab_dict={}
    # 1. sort the vocab
    sorted_vocab=sorted(vocab_dict.items(), key=lambda item:item[1],reverse=True)
    
    # 2. build the index 
    UNKnown_index=1
    index_vocab_dict[UNKnown_index]='UNKnown'
    index=2
    for word,frequency in sorted_vocab:
        if frequency>min_frequency:
            vocab_index_dict[word]=index
            index_vocab_dict[index]=word
            index+=1
        else:
            vocab_index_dict[word]=UNKnown_index
    print("words num in vocab:",index-1)
    return vocab_index_dict,index_vocab_dict
        
def encode_sentence(sent,vocab_index_dict):
    codes=[]
    for word in sent:
        index=vocab_index_dict[word]
        codes.append(index)
    assert len(codes)==len(sent)
    return codes

def output_one_set(src,tgt,file):
    assert len(src)==len(tgt)
    with open(file,'w') as fout:
        for i in range(len(src)):
            src_sent=src[i]
            pair=tgt[i]
            tgt_sent=pair['line']
            tgt_pid=pair['pid']
            src_sent=encode_sentence(src_sent,vocab_index_dict)
            tgt_sent=encode_sentence(tgt_sent,vocab_index_dict)
            fout.write(str(tgt_pid)+' ')
            src_str=''
            for index in src_sent:
                src_str+=str(index)
                src_str+=' '
            src_str=src_str[:-1]
            tgt_str=''
            for index in tgt_sent:
                tgt_str+=str(index)
                tgt_str+=' '
            tgt_str=tgt_str[:-1]
            fout.write(src_str)
            fout.write('|')
            fout.write(str(tgt_pid)+' ')
            fout.write(tgt_str+'\n')
    print('finish',file,len(src),'')
            
def output_vocab_file(index_vocab_dict,file):
    with open(file,'w') as fout:
        for i in range(1,len(index_vocab_dict)+1):
            word=index_vocab_dict[i]
            fout.write(word+'\n')    
    print(len(index_vocab_dict),'words saved in',file)            
    
    

In [18]:
# read files

vocab_dict={}
file=input_path+'/src-test.txt'
src_test,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-train.txt'
src_train,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-val.txt'
src_val,vocab_dict=read_src_data(file,vocab_dict)



file=input_path+'/src-test-full.txt'
src_test_full,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-val-full.txt'
src_val_full,vocab_dict=read_src_data(file,vocab_dict)


file=input_path+'/tgt-test.txt'
tgt_test,vocab_dict=read_tgt_data(file,vocab_dict)
file=input_path+'/tgt-test-full.txt'
tgt_test_full,vocab_dict=read_tgt_data(file,vocab_dict)

file=input_path+'/tgt-val.txt'
tgt_val,vocab_dict=read_tgt_data(file,vocab_dict)
file=input_path+'/tgt-val-full.txt'
tgt_val_full,vocab_dict=read_tgt_data(file,vocab_dict)

file=input_path+'/tgt-train.txt'
tgt_train,vocab_dict=read_tgt_data(file,vocab_dict)

In [26]:
# 2. build vocab

vocab_index_dict,index_vocab_dict=build_vocab(vocab_dict,min_frequency=4)

words num in vocab: 49844


In [21]:
# 3. output file

output_one_set(src_test,tgt_test,file='../data/src_data_full_feat_tf_resplited_review/test.txt')

output_one_set(src_train,tgt_train,file='../data/src_data_full_feat_tf_resplited_review/train.txt')

output_one_set(src_val,tgt_val,file='../data/src_data_full_feat_tf_resplited_review/valid.txt')

output_vocab_file(index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review/vocabulary',)


finish ../data/src_data_full_feat_tf_resplited_review/test.txt 4167 
finish ../data/src_data_full_feat_tf_resplited_review/train.txt 960114 
finish ../data/src_data_full_feat_tf_resplited_review/valid.txt 4167 
42182 words saved in ../data/src_data_full_feat_tf_resplited_review/vocabulary


In [22]:
output_one_set(src_val,tgt_val,file='../data/src_data_full_feat_tf_resplited_review/valid_testing.txt')

finish ../data/src_data_full_feat_tf_resplited_review/valid_testing.txt 4167 


In [ ]:
# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 1 --eval_steps 1 --layers 1 --no_save


# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 1 --eval_steps 1 --layers 1 --dev_file 'valid_testing.txt'  --no_save


# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 16 --max_iter 300000 --layers 3 --no_save

CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' --SpeakerMode \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114

CUDA_VISIBLE_DEVICES=1 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114



CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' \
--batch_size 10 --layers 3 --epochs 5 --eval_steps 2 --save_steps 2 --train_file 'valid_testing.txt'

## test encoding

In [6]:

def output_one_set(src,tgt,file):
    assert len(src)==len(tgt)
    with open(file,'w') as fout:
        for i in range(len(src)):
            src_sent=src[i]
            pair=tgt[i]
            tgt_sent=pair['line']
            tgt_pid=pair['pid']
            print(tgt_sent)
            src_sent=encode_sentence(src_sent,vocab_index_dict)
            tgt_sent=encode_sentence(tgt_sent,vocab_index_dict)
            fout.write(tgt_pid+' ')
            src_str=''
            for index in src_sent:
                src_str+=str(index)
                src_str+=' '
            src_str=src_str[:-1]
            tgt_str=''
            for index in tgt_sent:
                tgt_str+=str(index)
                tgt_str+=' '
            tgt_str=tgt_str[:-1]
            fout.write(src_str)
            fout.write('|')
            fout.write(tgt_pid+' ')
            fout.write(tgt_str+'\n')
            print(tgt_str)
    print('finish',file,len(src),'')

In [7]:
vocab_index_dict['telus']

1

In [8]:
output_one_set(src_val,tgt_val,file='../data/src_data_full_feat_tf_resplited_review/valid.txt')


['beach', 'is', 'better']
1853 15 151
['yeah', 'i', 'rest', 'my', 'mouse', 'on', 'top', 'of', 'the', 'arrow', 'keys', 'upside', 'down', ',', 'applying', 'the', 'right', 'amount', 'of', 'pressure', 'on', 'certain', 'areas', 'will', 'give', 'me', 'the', 'desired', 'effect', '.', 'i', 'slide', 'my', 'finger', 'across', 'the', 'sensor', 'to', 'aim']
567 11 437 27 3198 28 441 13 4 6530 2549 4074 159 6 4794 4 138 548 13 2102 28 723 2306 90 186 47 4 7982 1184 2 11 3909 27 1403 808 4 13211 10 4564
['like', 'mama', 'june', 'after', 'hot', 'yoga']
43 3947 5938 130 442 5104
['my', 'work', 'has', 'telus', 'internet', 'digit', 'paid', 'for', 'them', 'by', 'head', 'office', '.', 'that', "'", 's', 'a', 'pretty', 'bad', 'internet', 'package', '.']
27 127 91 1 368 21 666 23 72 82 329 668 2 17 7 18 9 209 163 368 3717 2
['i', "'", 'll', 'just', 'use', 'cheat', 'codes']
11 7 181 55 155 1967 5617
['had', 'the', 'weirdest', 'crush', 'on', 'this', 'really', 'religious', 'guy', 'in', 'high', 'school', '.', 'i

In [18]:
def build_vocab(file):
    d={}
    with open(file,'r') as fin:
        lines = fin.readlines()
        i=1
        for line in lines:
            line=line.strip()
            d[i]=line
            i+=1
    return d

In [19]:
index_dict=build_vocab('../data/movie/vocabulary')

In [22]:
line='1 6 1790 603 22 104 1562 10 3114 104 2|2 4 242 13 8207 6 13492 104 132 1 3114 2'
line='1 158 1590 9 668|2 6 10816 3953 22 487 10 2863 3953 2'
def decode(line,d):
    s,t=line.split('|')
    s=[int(i) for i in s.split(' ')]
    t=[int(i) for i in t.split(' ')]
    src_sent=[]
    tgt_sent=[]
    for index in s:
        src_sent.append(d[index])
    for index in t:
        tgt_sent.append(d[index])
    return src_sent,tgt_sent 


In [23]:
decode(line,d=index_dict)

(['UNknown', 'because', 'according', 'to', 'charlie'],
 ['.', 'the', 'surveillance', 'tape', 'is', 'also', 'a', 'sex', 'tape', '.'])

In [24]:
index_dict

{1: 'UNknown',
 2: '.',
 3: ',',
 4: 'you',
 5: 'i',
 6: 'the',
 7: '?',
 8: '-',
 9: 'to',
 10: 'a',
 11: "'s",
 12: 'it',
 13: "'t",
 14: '!',
 15: 'and',
 16: 'that',
 17: '...',
 18: 'of',
 19: 'in',
 20: 'me',
 21: 'what',
 22: 'is',
 23: 'he',
 24: 'we',
 25: 'for',
 26: 'this',
 27: 'my',
 28: 'your',
 29: 'be',
 30: 'have',
 31: 'do',
 32: 'don',
 33: 'on',
 34: "'ll",
 35: 'no',
 36: 'all',
 37: 'not',
 38: 'but',
 39: "'m",
 40: 'with',
 41: 'was',
 42: "'re",
 43: 'are',
 44: 'can',
 45: 'here',
 46: 'know',
 47: 'there',
 48: 'him',
 49: '',
 50: 'get',
 51: 'they',
 52: '"',
 53: 'if',
 54: 'go',
 55: 'like',
 56: 'now',
 57: 'she',
 58: 'so',
 59: 'come',
 60: 'well',
 61: 'out',
 62: 'at',
 63: 'up',
 64: 'right',
 65: 'yes',
 66: "'ve",
 67: 'just',
 68: 'about',
 69: 'will',
 70: 'one',
 71: ':',
 72: 'oh',
 73: 'how',
 74: 'her',
 75: 'see',
 76: 'as',
 77: 'good',
 78: 'got',
 79: 'why',
 80: 'want',
 81: 'think',
 82: 'his',
 83: 'let',
 84: "'d",
 85: 'did',
 86: '